In [3]:
import numpy as np

In [6]:
def mod_pert_random(low, likely, high, confidence=4, samples=30):
    """Produce random numbers according to the 'Modified PERT' 
    distribution. 
    
    :param low: The lowest value expected as possible.
    
    :param likely: The 'most likely' value, statistically, the mode.
    :param high: The highest value expected as possible.
    :param confidence: This is typically called 'lambda' in literature 
                        about the Modified PERT distribution. The value
                        4 here matches the standard PERT curve. Higher
                        values indicate higher confidence in the mode.
                        
    Formulas from "Modified Pert Simulation" by Paulo Buchsbaum.
    """
    # Check minimum & maximum confidence levels to allow:
    confidence = min(8, confidence)
    confidence = max(2, confidence)

    mean = (low + confidence * likely + high)/(confidence + 2)

    a = (mean - low)/(high - low) * (confidence + 2)
    b = ((confidence + 1) * high - low - confidence * likely) / (high - low)
    
    beta = np.random.beta(a, b, samples)
    beta = beta * (high - low) + low
    return beta

In [7]:
class Queue:
    def __init__(self):
        self.groups=[]
    
    def queue_size(self):
        return len(self.groups)
    
    def isEmpty(self):
        if len(self.groups)> 0:
            return False
        else:
            
            return True
    
    def add_queue(self, group):
        if group.get_vip():   # If current group is a VIP group, move it forward by four groups,
            if len(self.groups) > 4:
                for i in range(0,4):
                    if self.groups[i].get_vip():
                        self.groups.insert(i,group)
                        break
                    else:
                        self.groups.insert(4,group)
            elif len(self.groups) > 1 and len(self.groups) < 4:
                for i in range(0,len(self.groups)):
                    if self.groups[i].get_vip():
                        self.groups.insert(i,group)
                        break
                    else:
                        self.groups.insert(1,group)
        else:
            self.groups.insert(0,group)
            
    
    def del_queue(self): #delete last=delete first come group 
        return self.groups.pop()  

In [8]:
# this class can be regarded as "class printer" in that case.
class Table:
    def __init__(self, num, size):
        self.num= num # No. of the table
        self.size = size # Size of the table: for group of up to 2, 4 or 6.
        
        self.currentGroup = None # Is the table occupied or not.
        
    def busy(self):
        if self.currentGroup != None:
            return True
        else:
            return False      
    
    def startNext(self, newGroup):
        self.currentGroup = newGroup

In [9]:
class Group:
    def __init__(self, time, size, vip):
        self.timestamp = time # Time when group registered (entered into the queue)
        self.size = size #randomly define size from 1 - 6
        self.vip = vip # Whether the group is a vip group
        self.table= None # Which table the group will be assigned to
        self.timeRequest = mod_pert_random(0, 45, 120, samples=1).astype(int)  #How long will the group spend
    
    def get_stamp(self):
        return self.timestamp
    
    def get_size(self):
        return self.size
    
    
    def wait_time(self, current_time):
        return current_time - self.timestamp
    
    def get_vip(self):
        return self.vip
    
    def get_time_request(self): 
        return self.timeRequest

In [10]:
def tablesSetting(number_tables_2, number_tables_4, number_tables_6):
    table_2_list = []
    table_4_list = []
    table_6_list = [] 
    for i in range(number_tables_2):
        table_2_list.append(Table(i,2))
    
    for i in range(number_tables_4):
        table_4_list.append(Table(i+number_tables_2,4))
    
    for i in range(number_tables_6):
        table_6_list.append(Table(i+number_tables_4+number_tables_2,6))
    
    return (table_2_list, table_4_list, table_6_list)

In [11]:
"""
def generation(Duration):
    size = np.random.randint(1, 7, 30)
    vip = []
    for i in range(len(size)):
        num = np.random.randint(0, 101, 1)
        if (num >= 0) & (num <= 8):
            vip.append(True)
        else:
            vip.append(False)
    timestamp_list = mod_pert_random(0, 120, 240).astype(int)
    counter = 0
    queue_2 = Queue()
    queue_4 = Queue()
    queue_6 = Queue()
    
    table_2, table_4, table_6 = tablesSetting(6, 4, 2)
    
    for i in range(Duration):
        #print(i,'/n')
        if i in timestamp_list:
            #print(size[counter],'**')
            if size[counter] == 1 or size[counter] == 2:
                queue_2.add_queue(Group(i, 2, vip[counter]))
                counter += 1
            elif size[counter] == 3 or size[counter] == 4:
                queue_4.add_queue(Group(i, 4, vip[counter]))
                counter += 1
            elif size[counter] == 5 or size[counter] == 6:
                queue_6.add_queue(Group(i, 6, vip[counter]))
                counter += 1
    
    nextGroup_endTime_2 = []
    nextGroup_endTime_4 = []  
    nextGroup_endTime_6 = []  
    total_timeR_2 = []
    total_timeR_4 = []
    total_timeR_6 = []

    for i in range(Duration):
        if (i == queue_2.groups[-1].timestamp) and (not queue_2.isEmpty()):
            for t in table_2:
                if (not t.busy()) and (not queue_2.isEmpty()):
                    nextGroup = queue_2.del_queue()
                    nextGroup_endTime_2.append(nextGroup.timeRequest + 2) #I am not sure about this one 
                    t.startNext(nextGroup)
                    #2 min for table cleaning, rearragement and the time to walk in blabla
                    total_timeR_2.append(nextGroup.timestamp + nextGroup.timeRequest+2)
                    print('For 2-party table,','Table', t.num, 'is ready for next group',
                          'they waited', int(sum(total_timeR_2)/len(total_timeR_2)),'minutes in total.'
                          'There are still',queue_2.queue_size(),'groups are waiting.')
    
        if (i == queue_4.groups[-1].timestamp) and (not queue_4.isEmpty()):
            for t in table_4:
                if (not t.busy()) and (not queue_4.isEmpty()):
                    nextGroup = queue_4.del_queue()
                    nextGroup_endTime_4.append(nextGroup.timestamp + nextGroup.timeRequest + 2)
                    t.startNext(nextGroup)
                    #2 min for table cleaning, rearragement and the time to walk in blabla
                    total_timeR_4.append(nextGroup.timeRequest+2)
                    print('For 4-party table,','Table', t.num, 'is ready for next group',
                       'they waited', int(sum(total_timeR_4)/len(total_timeR_4)),'minutes in total.'
                          'There are still',queue_4.queue_size(),'groups are waiting.')

        if (i == queue_6.groups[-1].timestamp) and (not queue_6.isEmpty()):
            for t in table_6:
                if (not t.busy()) and (not queue_6.isEmpty()):
                    nextGroup = queue_6.del_queue()
                    nextGroup_endTime_6.append(nextGroup.timestamp + nextGroup.timeRequest + 2)
                    t.startNext(nextGroup)
                    #2 min for table cleaning, rearragement and the time to walk in blabla
                    total_timeR_6.append(nextGroup.timeRequest+2)
                    print('For 6-party table,','Table', t.num, 'is ready for next group',
                             'they waited', int(sum(total_timeR_6)/len(total_timeR_6)),'minutes in total.'
                          'There are still',queue_6.queue_size(),'groups are waiting.')
            
                
    #     for i in range(len(queue_2.queue_size())):
    #     nextGroup_endTime = {}
    #     
    #     if i in nextGroup_endTime_2.values():
    #         table_2[对应nextGroup_endTime_2值为i的keys].currentGroup = None  
    #         #if at least one of tables2 finish servicing, set currentGroup to None (this table is not busy)
    #         
    #     
    #     total_timeR = []
    #     
    #     for t in table_2:
    #         if (not t.busy()) and (not queue_2.isEmpty()):
    #             nextGroup = queue_2.del_queue()
    #             nextGroup_endTime_2[t] = i+nextGroup.timeRequest+2  #2min for table cleaning, rearragement and the time to walk in blabla
    #             total_timeR.append(nextGroup.timeRequest+2)
    #     
    # 
    #     
    # #return (queue_2, queue_4, queue_6)
"""

"\ndef generation(Duration):\n    size = np.random.randint(1, 7, 30)\n    vip = []\n    for i in range(len(size)):\n        num = np.random.randint(0, 101, 1)\n        if (num >= 0) & (num <= 8):\n            vip.append(True)\n        else:\n            vip.append(False)\n    timestamp_list = mod_pert_random(0, 120, 240).astype(int)\n    counter = 0\n    queue_2 = Queue()\n    queue_4 = Queue()\n    queue_6 = Queue()\n    \n    table_2, table_4, table_6 = tablesSetting(6, 4, 2)\n    \n    for i in range(Duration):\n        #print(i,'/n')\n        if i in timestamp_list:\n            #print(size[counter],'**')\n            if size[counter] == 1 or size[counter] == 2:\n                queue_2.add_queue(Group(i, 2, vip[counter]))\n                counter += 1\n            elif size[counter] == 3 or size[counter] == 4:\n                queue_4.add_queue(Group(i, 4, vip[counter]))\n                counter += 1\n            elif size[counter] == 5 or size[counter] == 6:\n                queu

In [122]:
output=generation(240)

[2 5 1 2 6 5 4 4 5 1 4 4 6 6 3 1 1 5 5 6 3 5 4 6 6 5 4 6 2 1]
Wait time: 0
Table No. 0
nextGroup_endTime: {<__main__.Table object at 0x114535e10>: array([92])}
Wait time: 0
Table No. 0
nextGroup_endTime: {<__main__.Table object at 0x114535e10>: array([127])}
Wait time: 0
Table No. 0
nextGroup_endTime: {<__main__.Table object at 0x114535e10>: array([93])}
0
Wait time: 0
Table No. 0
nextGroup_endTime: {<__main__.Table object at 0x114535e10>: array([120])}
0
Wait time: 0
Table No. 0
nextGroup_endTime: {<__main__.Table object at 0x114535e10>: array([170])}
0


In [90]:
output[0].groups

[]

In [121]:
def generation(Duration):
    size = np.random.randint(1,7,30)
    print(size)
    
    vip = []
    for i in range(len(size)):
        num = np.random.randint(0, 101, 1)
        if (num >= 0) & (num <= 8):
            vip.append(True)
        else:
            vip.append(False)
    timestamp_list = mod_pert_random(0, 120, 240).astype(int)
    counter = 0
    queue_2 = Queue()
    queue_4 = Queue()
    queue_6 = Queue()
    
    
    table_2,table_4,table_6=tablesSetting(6,4,2)
    
    total_timeR = [0]
    nextGroup_endTime_2={}
    
    # Add every group into corresponding queue
    for i in range(Duration):
        if i in timestamp_list:
            if size[counter] == 1 or size[counter] == 2:
                queue_2.add_queue(Group(i, 2, vip[counter]))
                counter += 1
            elif size[counter] == 3 or size[counter] == 4:
                queue_4.add_queue(Group(i, 4, vip[counter]))
                counter += 1
            elif size[counter] == 5 or size[counter] == 6:
                queue_6.add_queue(Group(i, 6, vip[counter]))
                counter += 1
                
                
        
        
        if i in nextGroup_endTime_2.values():
            for num in nextGroup_endTime_2.keys():
                print(num.num)
                if i == nextGroup_endTime_2[num]:
                    table_2[num.num].currentGroup = None
            #if at least one of tables2 finish servicing, set currentGroup to None (this table is not busy)
            
        
        

        for t in table_2:
             if (not t.busy()) and (not queue_2.isEmpty()):
                 nextGroup = queue_2.del_queue()
                 
                 print('Wait time:', nextGroup.wait_time(i))
                 nextGroup.table=t.num
                 print('Table No.', t.num)
                 
                 nextGroup_endTime_2[t] = i+nextGroup.timeRequest+2  #2min for table cleaning, rearragement and the time to walk in blabla
                 print('nextGroup_endTime:',nextGroup_endTime_2)
                 total_timeR.append(nextGroup.timeRequest+2)
        
    avg_timeR = sum(total_timeR)/len(total_timeR)
        
    return (queue_2, queue_4, queue_6)